# Exercise 5

Using the files *transition_probabilities.csv* and *ovservation_likelihoods.csv*, compute the likelihood of POS tags for the following word sequence:

    I want to race

## Task 1

Load the given files and:

a) Write a function to calculate all possible tag sequences for the given word sentence.

b) Write a function to select the most probable tag sequence.

c) Discuss your solution.

## Task 2

Load the given files and follow the algorithm shown in slide 13 of our Lab Session 5 to:

a) Compute the initialization step of Viterbi.

b) Compute the recursion step of Viterbi.

c) Compute termination step of Viterbi.